In [7]:


#  IMPORTS
import pandas as pd
import numpy as np
import joblib
import ta
import ccxt
from datetime import timedelta

from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

#  PATHS & CONFIG
CSV_PATH = r"C:\Users\krish\OneDrive\Desktop\BITCOIN FINAL FOLDER FOR PREDICTION!\testing!.csv"
MODEL_PATH = r"C:\Users\krish\OneDrive\Desktop\BITCOIN FINAL FOLDER FOR PREDICTION!\btc_ml_oos_model_by_krish.pkl"

SYMBOL = "BTC/USD"
TIMEFRAME = "1d"


# STEP 1: UPDATE CSV WITH LATEST BTC DATA

console.print("\n🔄 Updating BTC price data...", style="bold cyan")

df_existing = pd.read_csv(CSV_PATH)
df_existing["Date"] = pd.to_datetime(df_existing["Date"], format="%d-%m-%Y")

last_date = df_existing["Date"].max()
console.print(f"Last date in CSV: {last_date.date()}", style="dim")

exchange = ccxt.binance({
    "enableRateLimit": True,
    "options": {"defaultType": "spot"}
})


since = int((last_date + timedelta(days=1)).timestamp() * 1000)

ohlcv = exchange.fetch_ohlcv(
    SYMBOL,
    timeframe=TIMEFRAME,
    since=since,
    limit=1000
)

if ohlcv:
    df_new = pd.DataFrame(
        ohlcv,
        columns=["Timestamp", "Open", "High", "Low", "Close", "Volume"]
    )

    df_new["Date"] = pd.to_datetime(df_new["Timestamp"], unit="ms")
    df_new["Date"] = df_new["Date"].dt.strftime("%d-%m-%Y")
    df_new = df_new[["Date", "Open", "High", "Low", "Close", "Volume"]]

    df_existing["Date"] = df_existing["Date"].dt.strftime("%d-%m-%Y")
    df_final = pd.concat([df_existing, df_new], ignore_index=True)
    df_final.to_csv(CSV_PATH, index=False)

    console.print(f"✅ Added {len(df_new)} new day(s)", style="bold green")
else:
    console.print("ℹ️ No new BTC data available", style="yellow")


# STEP 2: LOAD MODEL

model = joblib.load(MODEL_PATH)
console.print("📦 Model loaded successfully", style="bold green")


# STEP 3: LOAD & CLEAN DATA

btc = pd.read_csv(CSV_PATH)
btc.columns = btc.columns.str.strip()

btc["Date"] = pd.to_datetime(btc["Date"], dayfirst=True)
btc.set_index("Date", inplace=True)
btc.sort_index(inplace=True)

numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
for col in numeric_cols:
    btc[col] = pd.to_numeric(btc[col], errors="coerce")

btc.dropna(subset=numeric_cols, inplace=True)


# STEP 4: FEATURE ENGINEERING (UNCHANGED)

btc['ret_1d'] = btc['Close'].pct_change()
btc['ret_5d'] = btc['Close'].pct_change(5)
btc['ret_20d'] = btc['Close'].pct_change(20)

btc['vol_10d'] = btc['ret_1d'].rolling(10).std()
btc['vol_30d'] = btc['ret_1d'].rolling(30).std()

btc['rsi'] = ta.momentum.RSIIndicator(btc['Close'], 14).rsi()

btc['ema_fast'] = ta.trend.EMAIndicator(btc['Close'], 20).ema_indicator()
btc['ema_slow'] = ta.trend.EMAIndicator(btc['Close'], 50).ema_indicator()

btc['trend'] = (btc['ema_fast'] - btc['ema_slow']) / btc['ema_slow']
btc['price_vs_ema'] = (btc['Close'] - btc['ema_slow']) / btc['ema_slow']
btc['bull_regime'] = (btc['Close'] > btc['ema_slow']).astype(int)

btc.dropna(inplace=True)


# STEP 5: PREDICTION

FEATURES = [
    'ret_5d',
    'ret_20d',
    'vol_10d',
    'vol_30d',
    'rsi',
    'trend',
    'price_vs_ema',
    'bull_regime'
]

latest = btc.iloc[-1:][FEATURES]
prob = model.predict_proba(latest)[0, 1]


# STEP 6: BEAUTIFUL USER-FRIENDLY OUTPUT (RICH)

today = latest.index[0].date()
regime = "Bullish" if latest['bull_regime'].iloc[0] == 1 else "Bearish"

if prob >= 0.70:
    confidence = "High"
elif prob >= 0.55:
    confidence = "Moderate"
else:
    confidence = "Low"

if prob >= 0.65 and latest['bull_regime'].iloc[0] == 1:
    action = "HOLD / ENTER LONG 🟢"
    action_style = "bold green"
    reasons = [
        "Model probability above 65% conviction threshold",
        "Price trading above long-term EMA (bullish regime)"
    ]
else:
    action = "STAY OUT 🔴"
    action_style = "bold red"
    reasons = [
        "Probability edge is insufficient",
        "Model prioritizes capital protection"
    ]

start = today + timedelta(days=1)
end = start + timedelta(days=4)

title = f"📊 BTC DAILY MODEL SUMMARY — {today}"

body = Text()
body.append(f"Market Regime        : {regime}\n", style="bold cyan")
body.append(f"5-Day Up Probability : {round(prob * 100, 1)}%\n")
body.append(f"Confidence Level     : {confidence}\n\n")

body.append("Recommended Action   : ", style="bold")
body.append(f"{action}\n\n", style=action_style)

body.append("Why this action?\n", style="bold underline")
for r in reasons:
    body.append(f" • {r}\n")

body.append(f"\nPrediction Window    : {start} → {end}\n", style="dim")

body.append(
    "\nNote:\n"
    "This is a probabilistic ML signal, not financial advice.\n"
    "No action is taken when conviction is low.",
    style="italic dim"
)

panel = Panel(
    body,
    title=title,
    border_style="blue",
    padding=(1, 2)
)

console.print("\n")
console.print(panel)
console.print("\n✅ DAILY RUN COMPLETED\n", style="bold green")


🔄 Updating BTC price data...

Last date in CSV: 2025-12-30

ℹ️ No new BTC data available

📦 Model loaded successfully

╭──────────────────────────────────── 📊 BTC DAILY MODEL SUMMARY — 2025-12-30 ────────────────────────────────────╮
│                                                                                                                 │
│  Market Regime        : Bearish                                                                                 │
│  5-Day Up Probability : 66.1%                                                                                   │
│  Confidence Level     : Moderate                                                                                │
│                                                                                                                 │
│  Recommended Action   : STAY OUT 🔴                                                                             │
│                                                                                                                 │
│  Why this action?                                                                                               │
│   • Probability edge is insufficient                                                                            │
│   • Model prioritizes capital protection                                                                        │
│                                                                                                                 │
│  Prediction Window    : 2025-12-31 → 2026-01-04                                                                 │
│                                                                                                                 │
│  Note:                                                                                                          │
│  This is a probabilistic ML signal, not financial advice.                                                       │
│  No action is taken when conviction is low.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ DAILY RUN COMPLETED